# MEGAnno+

This is a demo notebook for EACL 2024 System demonstration submission 'MEGAnno+: A Human-LLM Collaborative Annotation tool'.
MEGAnno+ makes it convenient and easy for users to annotate their datasets using LLMs, and also verify the LLM annotations. 

## Connecting to the service

We provide a shared project to simuliate the real-world collaboration among data science practitioners. This project is pre-loaded with a natural language inference (NLI) dataset ([link](https://github.com/alisawuffles/wanli)).

In order to use MegAnno+, we require an authentication to connect. For the purposes of the demo in the review process, we provide some authnetication keys for the reviewers to use. Note that token sharing is for the demostration purposes only, and might cause potential overwrite of other's annotation.

In [ ]:
demo_tok_1 = 'gAAAAABlW8yJ3hqiuMi-R-5zz5hAzFQLujQOS_w31REt9npE1hifSR80dVOIn1KGQ9pKNdXLOrjdti33B02sFFL6IpB0YVUARUomu9IsIjanLYTA3BPHfSduOrKrnynEgOc_nL6-4DY06Zeend0I6nAamuTjHTe5J173sJfMV_lbnFHi8RlPYL2X0U_iSrX4e9BCy55lfUkHgVRstdDuo1JeT0qbBCOcZN_la0pjgKWIms115T5Hn4AAgoTyPqV2KkGcHsctdbIc'
demo_tok_2 = 'gAAAAABlW8y8j-VhfeJ2s-nSE_qTtlqe4nXoMg8fjU_F6ZWXOJ8ZGAm_nKHsKNl_mP0objyn8dNArFfeBlY4Yie-x0Ojvml14Bu6N23TWjORrroV3PGGAKfXptZ8g5IMQM33eNdmcnnsjZOCV4Xc86nLm5qOLzJ0huw0OgDm5CpvbEvgefSIfiMUSo4urjixYr7W09EZhzBRZdBT99lMagF67Y3wcc-WPJ_XIyVjRFGYAxf928GQNWSC1G1OjUoe0CJk9TWbi085'
demo_tok_3 = 'gAAAAABlW8zpsp4e7LYt0cZe0WpITpyajEaURdGSIBgD3ALnIMJPYZ9Y3CN0oDFuza9BjJjufLfKfrfzS7DXaUj-CacBZeVIw9s6_EjtrMymvYVvSyRjHccUPWDvcje_ROgaO7zG5EaP9mMlzE2L2pQg1TXwy9dkjOQ6D07JByDqF91M5_Bt4_X692lx8Lkeqaa4lfAzSjqHM2yXs5gC2gOqNk9SPCJtz-1qLOdyGq-ttydeW4xs7EYNca3qTx97beO617G1ya4e'

In [ ]:
demo_token = demo_tok_1 # select any one demo token

In [ ]:
# connect to service
from labeler_client import Authentication, Service
demo_auth = Authentication(project='public_demo', access_token=demo_token)
service = Service(project='public_demo', token=demo_token)

## Use Case:
We now take you through the use case described in our paper. Moana is a Data Scientist working at a popular newspaper. Moana needs to train a model analyzing the degree of agreement between user comments and political opinion pieces — e.g., whether the comments entail the opinion. Moana needs to collect the training data quickly so decides to use MegAnno+.

### Set labeling schema
She sets the labeling schema for the task below as "entailment" and "not entailment"

In [ ]:
# set labeling schema
service.get_schemas().set_schemas({
    'label_schema': [
        {
            "name": "natural language inference",
            "level": "record",
            "options": [
                { "value": "entailment", "text": "entailment" },
                { "value": "not_entailment", "text": "not_entailment" },
            ]
        }
    ]
})

In [ ]:
# retrieve label schema
label_schema = service.get_schemas().value(active=True)[0]['schemas']['label_schema']
print(label_schema)

### Review data, (optional) perform human annotation
Moana reviews the dataset she wants annotated in our widget. Using the search capabilities and bulk annotation techniques of MegAnno, she knows she can annotate the dataset by herself too.

In [ ]:
# search results => subset
subset = service.search(limit=20)
# bring up a widget 
subset.show()

However, because she needs these annotations quickly without putting too much effort in at the moment. She decides to use LLM annotators.

### Model and prompt selection 

Moana chooses text-davinci-003 as her LLM and opts to go for the default prompt template provided by MegAnno+. She reviews it as follows:

In [ ]:
# select model configuration
model_config = {'model': 'text-davinci-003'}

In [ ]:
# set prompt template
from labeler_client import PromptTemplate
label_name = 'natural language inference'
template = PromptTemplate(label_schema, [label_name])
# open up a widget to preview
template.preview(records=['[sample input]', 'Premise: In a small city, a private company may not be able to provide enough of the basic services that a city needs.\nHypothesis: A private company cannot provide the basic services that a city needs.'])

### Get LLM Annotations 

Moana simply calls the `Controller` of MegAnno+ which prepares all the prompts, makes the call to the LLM API to obtain label responses, handles any errors that may have been encountered, and post-processes the LLM responses to store the necessary label and metadata (in this case, confidence scores) information. 

In [ ]:
# connect to controller
from labeler_client import Controller
controller = Controller(service, demo_auth)

In [ ]:
# create a new agent
agent_uuid = controller.create_agent(model_config, template)

In [ ]:
# select subset
subset = service.search(limit=20)
# run a LLM annotation job
job_uuid = controller.run_job(agent_uuid, subset, label_name)

### Human Verification

Moana now can take a look at the labels with which the dataset is annotated using the LLM. 

In [ ]:
# open a verification widget for the job
verf_subset = service.search_by_job(job_id=job_uuid, verified=False)
verf_subset.show({"title": "Verification", "view": "table", "mode": "verifying"})

## Iterations
Moana now decides to change the schema of the task. 

In [ ]:
# update labeling schema
service.get_schemas().set_schemas({
    'label_schema': [
        {
            "name": "natural language inference",
            "level": "record",
            "options": [
                { "value": "entailment", "text": "entailment" },
                { "value": "neutral", "text": "neutral" },
                { "value": "contradiction", "text": "contradiction" },
            ]
        }
    ]
})
label_schema2 = service.get_schemas().value(active=True)[0]['schemas']['label_schema']
print(label_schema)

She also considers utilizing a new agent and changing some of the LLM parameters such as temperature to 0.

In [ ]:
# select model configuration
model_config2 = {'model': 'text-davinci-003', 'temperature': 0}

She also previews the template and makes some customizations.

In [ ]:
# set prompt template
template2 = PromptTemplate(label_schema2, [label_name])
template2.preview(records=['[sample input]', 'Premise: In a small city, a private company may not be able to provide enough of the basic services that a city needs.\nHypothesis: A private company cannot provide the basic services that a city needs.'])

Moana registers a new agent and runs a new job with the same subset.

In [ ]:
# create a new agent
agent_uuid = controller.create_agent(model_config2, template2)

In [ ]:
# run a LLM annotation job
job_uuid = controller.run_job(agent_uuid, subset, label_name)

### Export
Labels collected by all annotators can be exported.

In [ ]:
# export all labels
service.export()